In [100]:
import pandas as pd

In [101]:
df = pd.read_parquet('../../data/curated/2022_vote_probs.parquet')

In [102]:
df

,match_id,match_round,player_id,player_first_name,player_last_name,player_team,0,1,2,3
76187,16117,1,11904,Tom,Liberatore,Western Bulldogs,0.998700,0.000973,0.000299,0.000028
76188,16117,1,11945,Steven,May,Melbourne,0.995085,0.003862,0.000924,0.000128
76189,16117,1,11972,Max,Gawn,Melbourne,0.940318,0.031202,0.023883,0.004596
76190,16117,1,12015,Tom,McDonald,Melbourne,0.998167,0.001388,0.000399,0.000046
76191,16117,1,12034,Adam,Tomlinson,Melbourne,0.999429,0.000466,0.000096,0.000008
...,...,...,...,...,...,...,...,...,...,...
85114,16346,23,12863,Logan,McDonald,Sydney,0.996589,0.002546,0.000776,0.000090
85115,16346,23,12864,Errol,Gulden,Sydney,0.982434,0.011877,0.004978,0.000710
85116,16346,23,12945,Mitch,Owens,St Kilda,0.997681,0.001821,0.000448,0.000050
85117,16346,23,12947,Nasiah,Wanganeen-Milera,St Kilda,0.996853,0.002366,0.000708,0.000073


In [103]:
match_ids = df.match_id.unique()

In [104]:
def standardise_votes(df):
    
    ini_sum_1 = sum(df['1'])
    ini_sum_2 = sum(df['2'])
    ini_sum_3 = sum(df['3'])
    
    df['new_3'] = df['3'] / ini_sum_3
    df['new_2'] = df['2'] / ini_sum_2
    df['new_1'] = df['1'] / ini_sum_1

    df['new_0'] = 1 - (df['new_1'] + df['new_2'] + df['new_3'])
    
    df['exp_votes'] = 1*df['new_1'] + 2*df['new_2'] + 3*df['new_3']
    
    
    return df[['player_id', 'match_id', 'player_first_name', 'player_last_name', 'player_team', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes']]

In [105]:
final_df = pd.DataFrame(columns=['player_id', 'match_id', 'player_first_name', 'player_last_name', 'player_team', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes'])


for idx in match_ids:
    
    df_match = df[df['match_id'] == idx].reset_index() 
    
    final_df = pd.concat([final_df, standardise_votes(df_match)])

In [106]:
final_df

,player_id,match_id,player_first_name,player_last_name,player_team,new_0,new_1,new_2,new_3,exp_votes
0,11904,16117,Tom,Liberatore,Western Bulldogs,0.998848,0.000859,0.000269,0.000025,0.001471
1,11945,16117,Steven,May,Melbourne,0.995647,0.003407,0.000831,0.000115,0.005415
2,11972,16117,Max,Gawn,Melbourne,0.946869,0.027525,0.021477,0.004129,0.082865
3,12015,16117,Tom,McDonald,Melbourne,0.998376,0.001224,0.000359,0.000042,0.002066
4,12034,16117,Adam,Tomlinson,Melbourne,0.999494,0.000411,0.000087,0.000008,0.000608
...,...,...,...,...,...,...,...,...,...,...
41,12863,16346,Logan,McDonald,Sydney,0.996991,0.002224,0.000666,0.000119,0.003913
42,12864,16346,Errol,Gulden,Sydney,0.984406,0.010376,0.004274,0.000944,0.021757
43,12945,16346,Mitch,Owens,St Kilda,0.997958,0.001591,0.000384,0.000066,0.002559
44,12947,16346,Nasiah,Wanganeen-Milera,St Kilda,0.997228,0.002067,0.000608,0.000097,0.003574


In [107]:
final_df.to_parquet('../../data/curated/adjusted_probabilities_2022.parquet')